In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/multi_turn_rewrite/chinese/data')

In [0]:
from pathlib import Path
from collections import Counter

import random

In [3]:
Path('../vocab').mkdir(exist_ok=True)
counter_char = Counter()

with open('corpus.txt') as f, open('train.txt', 'w') as f_train, open('test.txt', 'w') as f_test:
  for line in f:
    line = line.rstrip()
    try:
      h1, h2, q, a = [seg for seg in line.split('\t') if seg != '']
    except:
      print('Deleted:', line)
      continue
    if random.random() < 0.05:
      f_test.write(h1+'\t'+h2+'\t'+q+'\t'+a+'\n')
    else:
      f_train.write(h1+'\t'+h2+'\t'+q+'\t'+a+'\n')
    counter_char.update(list(h1)+list(h2)+list(q)+list(a))

most_common = lambda x: [w for w, freq in x.most_common()]
chars = ['<pad>', '<start>', '<end>'] + most_common(counter_char)
print(len(chars), 'Vocabulary')

with open('../vocab/char.txt', 'w') as f:
  for char in chars:
    f.write(char+'\n')

Deleted: 晚上需要开空调吗				回答我		回答我什么时候开始晴天
Deleted: 第五元素最喜欢的吕克贝松电影之一另一部是圣女贞德 				我看过这个电视剧了		我看过第五元素电视剧了
3878 Vocabulary
